In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [0]:
df=pd.read_hdf('data/car.h5')

In [5]:
df.shape

(106494, 155)

##Feature Engineering

In [0]:
SUFFIX_CAT='_cat'

for feat in df.columns:
  if isinstance(df[feat][0],list): continue

  factorized_values=df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat]=factorized_values
  else:
    df[feat + SUFFIX_CAT]=factorized_values  

In [11]:
cat_feats=[x for x in df.columns if SUFFIX_CAT in x]
cat_feats=[x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

##Decision Tree

In [0]:
def run_model(model,feats):
  x=df[feats].values
  y=df['price_value'].values
  scores = cross_val_score(model,x,y,cv=3,scoring='neg_mean_absolute_error')
  return np.mean(scores),np.std(scores)

In [35]:
run_model(DecisionTreeRegressor(max_depth=5),cat_feats)

(-19695.13091100928, 148.72570644015792)

##Random Forest

In [36]:
run_model(RandomForestRegressor(
    max_depth=5,n_estimators=50,random_state=0),cat_feats
    )

(-18718.657185256638, 64.5424578125788)

## XGBoostx

In [37]:
xgb_params={
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}
run_model(xgb.XGBRegressor(**xgb_params),cat_feats)

[20:47:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:47:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:48:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [38]:
m = xgb.XGBRegressor(max_depth=5,n_estimators=50,learning_rate=0.1,seed=0)
m.fit(x,y)

imp = PermutationImportance(m,random_state=0).fit(x,y)
eli5.show_weights(imp,feature_names=cat_feats)

[20:50:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd_cat
0.1132 ± 0.0032,param_rok-produkcji_cat
0.1090 ± 0.0025,param_stan_cat
0.0619 ± 0.0025,param_skrzynia-biegów_cat
0.0568 ± 0.0016,param_faktura-vat_cat
0.0489 ± 0.0014,param_moc_cat
0.0273 ± 0.0007,param_marka-pojazdu_cat
0.0242 ± 0.0013,feature_kamera-cofania_cat
0.0212 ± 0.0008,param_typ_cat
0.0174 ± 0.0008,param_pojemność-skokowa_cat


In [69]:
feats = ['param_napęd_cat',
'param_rok-produkcji', #zmiana na z kategorialnych na num
'param_stan_cat',
'param_skrzynia-biegów_cat',
'param_faktura-vat_cat',
'param_moc', #zmiana na z kategorialnych na num
'param_marka-pojazdu_cat',
'feature_kamera-cofania_cat',
'param_typ_cat',
'param_pojemność-skokowa', #zmiana na z kategorialnych na num
'seller_name_cat',
'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat',
'param_wersja_cat',
'param_kod-silnika_cat',
'feature_system-start-stop_cat',
'feature_asystent-pasa-ruchu_cat',
'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat',
'feature_regulowane-zawieszenie_cat']
len(feats)

20

In [73]:
run_model(xgb.XGBRegressor(**xgb_params),feats)

[21:47:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:47:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:47:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [0]:
df['param_rok-produkcji']=df['param_rok-produkcji'].map(lambda x:-1 if str(x)=='None' else int(x))

In [0]:
df['param_moc']=df['param_moc'].map(lambda x: -1 if str(x)=='None' else x.split(' ')[0])

In [0]:
df['param_pojemność-skokowa']=df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)=='None' else x.split('cm3')[0].replace(' ',''))

In [0]:
!git config --global user.email "jedrzejczak.wa@gmail.com"
!git config --global user.name "Wieslaw"

In [0]:
!git add day4.ipynb
!git commit -m "DecTree,RandomF,XGBoost"

In [0]:
!git push -u origin master